In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
!ls /home/pamessina/medvqa-workspace/cache/mimiccxr/openai/

 gpt-3.5-turbo-0301_comparisons__part1.jsonl
'gpt-3.5-turbo-0301_comparisons__part1(obsolete).jsonl'
 gpt-3.5-turbo-0301_comparisons__part2.jsonl
 gpt-3.5-turbo-0301_parsed_sentences.jsonl
 gpt-3.5-turbo-0613_comparisons__part1.jsonl
 gpt-3.5-turbo-0613_comparisons__part2.jsonl
 gpt-3.5-turbo-0613_paraphrased_anatomical_locations__part1.jsonl
 gpt-3.5-turbo-0613_paraphrased_anatomical_locations__part2.jsonl
 gpt-3.5-turbo-0613_paraphrased_observations__single-words.jsonl
 gpt-3.5-turbo-0613_paraphrased_observations__two-or-more-words__part1.jsonl
 gpt-3.5-turbo-0613_paraphrased_observations__two-or-more-words__part3.jsonl
 gpt-3.5-turbo-0613_parsed_facts__hard.jsonl
 gpt-3.5-turbo-0613_parsed_facts__uniform.jsonl
 gpt-3.5-turbo-0613_parsed_facts__v2.jsonl
'gpt-3.5-turbo-0613_parsed_facts__v2_offset=20000.jsonl'
'gpt-3.5-turbo-0613_parsed_facts__v2_offset=40000_uniform.jsonl'
 gpt-3.5-turbo-0613_parsed_sentences__v2.jsonl
'gpt-3.5-turbo-0613_parsed_sentences__v2(uniform).jsonl'
 gpt-3.5

In [55]:
!python ../../scripts/mimiccxr/extract_comparisons_from_facts_with_transformer.py \
    --integrated_fact_metadata_filepath \
        "/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(578733,58628071).jsonl" \
    --preprocessed_sentences_to_skip_filepaths \
        "/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_comparisons__part1.jsonl" \
        "/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_comparisons__part2.jsonl" \
    --checkpoint_folder_path \
        "/mnt/data/pamessina/workspaces/medvqa-workspace/models/seq2seq/20230718_103406_fact2comparison(gpt-3.5-.._part1;gpt-3.5-.._part2)_Seq2Seq(t5-small)/" \
    --logging_level "INFO" \
    --batch_size 1000 \
    --num_beams 1 \
    --max_length 50 \
    --num_workers 4

2023-07-18 11:38:44,533 - INFO - Loading preprocessed sentences to skip from /home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_comparisons__part1.jsonl
2023-07-18 11:38:44,601 - INFO - Loaded 13977 already processed sentences from /home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_comparisons__part1.jsonl
2023-07-18 11:38:44,601 - INFO - Loading preprocessed sentences to skip from /home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_comparisons__part2.jsonl
2023-07-18 11:38:44,841 - INFO - Loaded 44457 already processed sentences from /home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_comparisons__part2.jsonl
2023-07-18 11:38:44,841 - INFO - Loading facts metadata from /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(578733,58628071).jsonl
100%|███████████████████████████████| 578733/578733 [00:01<00:00, 380309.39it/s]
2023-07-18 11:38:51,728 - INFO - Found 476977 sentence

In [5]:
from medvqa.utils.files import load_jsonl

In [43]:
aux = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(578733,58628071).jsonl')

In [54]:
q = 'worsening osteoporosis in a debilitated and wheelchair-bound patient'
for row in aux:
    if row['fact'] == q:
        print(row)

{'fact': 'worsening osteoporosis in a debilitated and wheelchair-bound patient', 'metadata': {'anatomical location': '', 'detailed observation': 'worsening osteoporosis', 'short observation': 'osteoporosis', 'category': 'disease', 'health status': 'abnormal', 'prev_study_comparison?': 'no', 'comparison status': ''}, 'extraction_method': 'gpt-3.5-turbo-0613'}


In [44]:
paraphrased_inputs_jsonl_filepaths = [
    "/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_paraphrased_observations__two-or-more-words__part1.jsonl",
    "/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-16k-0613_paraphrased_observations__two-or-more-words__part2.jsonl",
    "/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_paraphrased_observations__two-or-more-words__part3.jsonl",
    "/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-16k-0613_paraphrased_observations__two-or-more-words__part4.jsonl",
]
input2paraphrases = {}
for filepath in paraphrased_inputs_jsonl_filepaths:
    paraphrased_inputs = load_jsonl(filepath)
    for row in paraphrased_inputs:
        input_text = next(iter(row['metadata'].values()))
        paraphrases = row['parsed_response']
        if input_text not in input2paraphrases:
            input2paraphrases[input_text] = []
        input2paraphrases[input_text].extend(paraphrases)

In [46]:
inputs = list(input2paraphrases.keys())

In [67]:
# x = 10
# x = inputs.index('degenerative bone disease worsening')
x = inputs.index('slight irregularity of the left lateral seventh rib')
inputs[x], input2paraphrases[inputs[x]]

ValueError: 'slight irregularity of the left lateral seventh rib' is not in list

In [52]:
q = 'degenerative bone disease worsening'
for x, p in input2paraphrases.items():
    if x == q or q in p:
        print((x, p))

('worsening osteoporosis in a debilitated and wheelchair-bound patient', ['deteriorating osteoporosis', 'progressive osteoporosis', 'advancing osteoporosis', 'worsening bone thinning', 'declining bone density', 'aggravating bone weakening', 'deterioration of bone strength', 'increasing fragility of bones', 'escalating loss of bone density', 'degenerative bone disease worsening', 'osteoporosis worsening in a debilitated and wheelchair-bound patient'])


In [56]:
tmp = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/extracted_comparisons_Seq2Seq(t5-small)_50_1_20230718_114030.jsonl')

In [57]:
len(tmp)

476977

In [58]:
tmp[-20:]

[{'sentence': 'tip of Swan-Ganz catheter in the pulmonary outflow tract',
  'comparison': 'no comparison'},
 {'sentence': 'volume loss potentially associated with a mild right pleural effusion',
  'comparison': 'decrease'},
 {'sentence': 'screws in place', 'comparison': 'no comparison'},
 {'sentence': 'significant improvement compared to the most recent prior exam',
  'comparison': 'improved'},
 {'sentence': 'stable linear opacification in the left mid lung',
  'comparison': 'stable/unchanged'},
 {'sentence': 'comparison with chest radiographs read in conjunction with chest CT',
  'comparison': 'resolved'},
 {'sentence': 'ETT tip terminates approximately 5.2 cm above the carina',
  'comparison': 'no comparison'},
 {'sentence': 'diffuse sclerosis of the bones', 'comparison': 'no comparison'},
 {'sentence': 'new opacification base of the left lung',
  'comparison': 'new finding'},
 {'sentence': 'trace pleural effusion in the left posterior costophrenic angle',
  'comparison': 'unclear co

In [59]:
!python ../../scripts/mimiccxr/integrate_metadata_and_comparisons.py \
    --integrated_fact_metadata_filepath \
        "/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(578733,58628071).jsonl" \
    --extracted_comparisons_filepaths \
        "/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_comparisons__part1.jsonl" \
        "/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_comparisons__part2.jsonl" \
        "/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/extracted_comparisons_Seq2Seq(t5-small)_50_1_20230718_114030.jsonl" \
    --extraction_methods \
        "gpt-3.5-turbo-0613" \
        "gpt-3.5-turbo-0613" \
        "t5-small-finetuned"

Loading extracted comparisons from /home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_comparisons__part1.jsonl...
Loading extracted comparisons from /home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_comparisons__part2.jsonl...
Loading extracted comparisons from /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/extracted_comparisons_Seq2Seq(t5-small)_50_1_20230718_114030.jsonl...
Loading extracted facts and metadata...
Updated 521435/578733 facts with comparisons.
no comparison: 283490
stable/unchanged: 62095
worsened: 27244
position changed: 23214
improved: 22099
resolved: 20332
unclear comparison: 19614
new finding: 17690
increase: 14539
decrease: 9433
progressed: 7418
smaller: 5337
larger: 5236
reappeared: 3548
other: 146
Saving updated extracted facts and metadata...
Saved updated extracted facts and metadata to /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(578733,58628071).improved_comparison(652629

In [60]:
!ls -lh '/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(578733,58628071).improved_comparison(6526297).jsonl'

-rw-rw-r-- 1 pamessina pamessina 265M Jul 18 11:46 '/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(578733,58628071).improved_comparison(6526297).jsonl'


In [61]:
tmp2 = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(578733,58628071).improved_comparison(6526297).jsonl')

In [62]:
len(tmp2)

578733

In [64]:
tmp2[-11]

{'fact': 'interval development of a small focal area of heterogeneously increased density in the anterior segment of the right upper lobe',
 'metadata': {'anatomical location': 'anterior segment of the right upper lobe',
  'detailed observation': 'interval development of a small focal area of heterogeneously increased density',
  'short observation': 'small focal area of increased density',
  'category': 'anatomical finding',
  'health status': 'abnormal',
  'prev_study_comparison?': 'yes',
  'comparison status': 'new'},
 'extraction_method': 't5-small-finetuned',
 'improved_comparison': {'comparison': 'new finding',
  'extraction_method': 't5-small-finetuned'}}